#### This assignment may be worked individually or in pairs. Enter your name/s here:
    

In [1]:
# Danica Padlan, Ashley Yude

#Don't need to do this
#Feature Engineering Notes
#-calc average of columns 2-7 for avg MA result detection
#-maybe calc avg of columns 8-15 for avg normalized exudate

# Assignment 2: Decision Trees

In this assignment we'll implement the Decision Tree algorithm to classify patients as either having or not having diabetic retinopathy. For this task we'll be using the Diabetic Retinopathy data set, which contains features from the Messidor image set to predict whether an image contains signs of diabetic retinopathy or not. This dataset has `1150` records and `20` attributes (some categorical, some continuous). You can find additional details about the dataset [here](http://archive.ics.uci.edu/ml/datasets/Diabetic+Retinopathy+Debrecen+Data+Set).

Attribute Information:

0) The binary result of quality assessment. 0 = bad quality 1 = sufficient quality.

1) The binary result of pre-screening, where 1 indicates severe retinal abnormality and 0 its lack. 

2-7) The results of MA detection. Each feature value stand for the number of MAs found at the confidence levels alpha = 0.5, . . . , 1, respectively. 

8-15) contain the same information as 2-7) for exudates. However, as exudates are represented by a set of points rather than the number of pixels constructing the lesions, these features are normalized by dividing the 
number of lesions with the diameter of the ROI to compensate different image sizes. 

16) The euclidean distance of the center of the macula and the center of the optic disc to provide important information regarding the patient's condition. This feature is also normalized with the diameter of the ROI.

17) The diameter of the optic disc. 

18) The binary result of the AM/FM-based classification.

19) Class label. 1 = contains signs of Diabetic Retinopathy, 0 = no signs of Diabetic Retinopathy.

#### Implementation: 
The function prototypes are given to you, please don't change those. You can add additional helper functions if needed. 

*Suggestion:* The dataset is substantially big, for the purpose of easy debugging, work with a subset of the data and test your decision tree implementation on that.

#### Notes:
Parts of this assignment will be **autograded** so a couple of caveats :-
- Entropy is calculated using log with base 2, `math.log2(x)`.
- For continuous features ensure that the threshold value lies exactly between 2 values. For example, if for feature 2 the best split occurs between 10 and 15 then the threshold value will be set as 12.5. For binary features [0/1] the threshold value will be 0.5.
- All values < `thresh_val` go to the left child and all values >= `thresh_val` go to the right child.

In [2]:
# Standard Headers
# You are welcome to add additional headers if you wish
# EXCEPT for scikit-learn... You may NOT use scikit-learn for this assignment!
import pandas as pd
import numpy as np
from math import log2

In [3]:
class DataPoint:
    def __str__(self):
        return "< " + str(self.label) + ": " + str(self.features) + " >"
    def __init__(self, label, features):
        self.label = label # the classification label of this data point
        self.features = features

Q1. Read data from a CSV file. Put it into a list of `DataPoints`.

In [4]:
def get_data(filename):
    data = []
    df = pd.read_csv(filename, header=None, names=['Quality Assessment Result', 'Pre-screening Result', 'MA Detection Result 1',
                                                  'MA Detection Result 2', 'MA Detection Result 3', 'MA Detection Result 4', 'MA Detection Result 5',
                                                  'MA Detection Result 6', 'Normalized Exudate 1', 'Normalized Exudate 2', 'Normalized Exudate 3',
                                                  'Normalized Exudate 4', 'Normalized Exudate 5', 'Normalized Exudate 6', 'Normalized Exudate 7',
                                                  'Normalized Exudate 8', 'Euclidean Distance', 'Optic Disc Diameter', 'AM/FM Result', 'Has Diabetic Retinopathy'])
    
    #loop through all rows in data frame
    for x in range (df.shape[0]):
        
        #create Data Point and add to list
        #before had 1:19
        new_DataPoint = DataPoint(df.iloc[x,19], df.iloc[x, 0:19])
        data.append(new_DataPoint)
    return data

In [5]:
class TreeNode:
    is_leaf = True          # boolean variable to check if the node is a leaf
    feature_idx = None      # index that identifies the feature
    thresh_val = None       # threshold value that splits the node
    prediction = None       # prediction class (only valid for leaf nodes)
    left_child = None       # left TreeNode (all values < thresh_val)
    right_child = None      # right TreeNode (all values >= thresh_val)
    
    def printTree(self, level=0):    # for debugging purposes
        if self.is_leaf:
            print ('-'*level + 'Leaf Node:      predicts ' + str(self.prediction))
        else:
            print ('-'*level + 'Internal Node:  splits on feature ' 
                   + str(self.feature_idx) + ' with threshold ' + str(self.thresh_val))
            self.left_child.printTree(level+1)
            self.right_child.printTree(level+1)

Q2. Implement the function `make_prediction` that takes the decision tree root and a `DataPoint` instance and returns the prediction label.

In [6]:
def make_prediction(tree_root, data_point):
    
    #base case
    if tree_root.is_leaf:
        return tree_root.prediction
    
    #check left/right node based on comparison to thresh value
    if (data_point.features.iloc[tree_root.feature_idx] < tree_root.thresh_val):
        return make_prediction(tree_root.left_child, data_point)
    else:
        return make_prediction(tree_root.right_child, data_point)

In [7]:
# #testing make predict
# data = get_data("messidor_features.txt")
# # print(len(data[0].features))

# #isDiabRetin = TreeNode(True, None, None, 1, None, None)
# isDiabRetin = TreeNode()
# isDiabRetin.is_leaf = True
# isDiabRetin.feature_idx = None      
# isDiabRetin.thresh_val = None       
# isDiabRetin.prediction = 1       
# isDiabRetin.left_child = None       
# isDiabRetin.right_child = None

# #notDiabRetin = TreeNode(True, None, None, 0, None, None)
# notDiabRetin = TreeNode()
# notDiabRetin.is_leaf = True
# notDiabRetin.feature_idx = None      
# notDiabRetin.thresh_val = None       
# notDiabRetin.prediction = 0       
# notDiabRetin.left_child = None       
# notDiabRetin.right_child = None

# #root = TreeNode(False, 0, 1, None, notDiabRetin, isDiabRetin)
# root = TreeNode()
# root.is_leaf = False
# root.feature_idx = 0 
# root.thresh_val = 1       
# root.prediction = None      
# root.left_child = isDiabRetin       
# root.right_child = notDiabRetin

# print(make_prediction(root, data[0]))

Q3. Implement the function `split_dataset` given an input data set, a `feature_idx` and the `threshold` for the feature. `left_split` will have all values < `threshold` and `right_split` will have all values >= `threshold`.

In [8]:
def split_dataset(data, feature_idx, threshold):
    left_split = []
    right_split = []
    
    #Loop through data and split based on comparison to threshold
    for x in range (len(data)):
        
        if (data[x].features.iloc[feature_idx] < threshold):
            left_split.append(data[x])
        else:
            right_split.append(data[x])
                    
    return (left_split, right_split)

Q4. Implement the function `calc_entropy` to return the entropy of the input dataset.

In [9]:
def calc_entropy(data):
    #data is an array
    
    #Special case
    if (len(data) == 0):
        return 0
    
    entropy = 0.0
    yes_count = 0
    no_count = 0
    
    #Increment yes & no counts for each datapoint's label
    for cur_datapoint in data:
        
        if cur_datapoint.label == 1:
            yes_count+= 1
        else:
            no_count+= 1
    
    #Create fractions based on yes/no counts
    yes_frac = (yes_count / len(data)) if yes_count > 0 else 0
    no_frac = (no_count / len(data)) if no_count > 0 else 0
    
    #Calculate Entropy
    if (yes_frac == 0):
        entropy = - ((no_frac)*log2(no_frac))
    elif (no_frac == 0):
        entropy = - ((yes_frac)*log2(yes_frac))
    else:
        entropy = -((yes_frac)*log2(yes_frac)) - ((no_frac)*log2(no_frac))
    
    return abs(entropy)

In [10]:
# # testing calc_entropy
# # 2, 1, 3, 7, 10, 11
# temp = []
# data = get_data("messidor_features.txt")
# temp.append(data[2])
# temp.append(data[1])
# temp.append(data[3])
# temp.append(data[7])
# temp.append(data[10])
# temp.append(data[11])

# temp = np.array(temp)

# print(calc_entropy(temp))

Q5. Implement the function `calc_best_threshold` which returns the best information gain and the corresponding threshold value for one feature at `feature_idx`.

In [11]:
def calc_best_threshold(data, feature_idx):
    best_info_gain = 0.0
    best_thresh = 0.0
    
    #keep track of datapoints in ascending order of feature values
    #take in values of specific feature in data list
    #[feature sorted by, class label]
    rows, cols = (len(data), 2)
    
    #creates matrix
    sorted_features = [ [0]*cols for i in range(rows)]
    
    #fills in matrix by feature value by feature index and class label
    for x in range (len(data)): 
        sorted_features[x][0] = data[x].features.iloc[feature_idx]
        sorted_features[x][1] = data[x].label
    
    #sorted by feature values in ascending order
    
    sorted_features = np.array(sorted_features)
    sorted_features = sorted_features[sorted_features[:,0].argsort()]
    
    #loop and find best threshold combo
    for x in range (len(sorted_features) - 1):
        
        #checks if data next to each other has different labels
        if sorted_features[x][1] != sorted_features[x+1][1]: 
            
            #also works for binary values!
            #main problem is threshold isn't ~X.5, we get a whole number
            mid_thresh = (sorted_features[x][0] + sorted_features[x+1][0]) / 2
            
            #use split method and pass threshold 
            l_and_r = split_dataset(data, feature_idx, mid_thresh)
            
            left = l_and_r[0]
            right = l_and_r[1]
            
            #get entropy values for parent, left, and right data
            left_ent = calc_entropy(left)
            right_ent = calc_entropy(right)
            parent_ent = calc_entropy(data)
            
            #calculate class labels from both splits  by sending it in to calc entropy
            cur_gain = parent_ent - (((len(left) / len(data)) * left_ent) + ((len(right) / len(data)) * right_ent)) 
            
            #checking for highest gain 
            if cur_gain > best_info_gain:
                best_info_gain = cur_gain
                best_thresh = mid_thresh
    
    return (best_info_gain, best_thresh)

In [12]:
#tested!
#testing calc_best_threshold()
# feat = 2;
# test_data = []
# test_data.append(data[0])
# test_data.append(data[1])
# test_data.append(data[2])
# test_data.append(data[3])
# test_data.append(data[4])
# test_data.append(data[5])
# result = calc_best_threshold(test_data, feat)
# print(result)


Q6. Implement the function `identify_best_split` which returns the best feature to split on for an input dataset, and also returns the corresponding threshold value.

In [13]:
def identify_best_split(data):
    
    #special case 
    if len(data) < 2:
        return (None, None)
    best_feature = 0.0
    best_thresh = 0.0
    best_gain = 0.0

    #loop through all features
    for x in range (len(data[0].features)):
        
        #get high gain and best thresh
        cur_gain, cur_thresh = calc_best_threshold(data, x)

        #save thresh and feature based on highest gain
        if cur_gain > best_gain:
            best_gain = cur_gain
            best_thresh = cur_thresh
            best_feature = x

    #return the one with best gain
    return (best_feature, best_thresh)

In [14]:
#tested!
#testing identify_best_split
# feat = 2;
# test_data = []
# test_data.append(data[0])
# test_data.append(data[1])
# test_data.append(data[2])
# test_data.append(data[3])
# test_data.append(data[4])
# test_data.append(data[5])
# result = calc_best_threshold(test_data, feat)
# print(result)
# identify_best_split(data)

Q7. Implement the function `create_leaf_node` which returns a `TreeNode` with `is_leaf=True` and `prediction` set to whichever classification occurs most in the dataset at this node.

In [15]:
def create_leaf_node(data):
    
    # Making a leaf node & filling in variables
    leaf = TreeNode()
    leaf.is_leaf = True
    
    yes = 0
    no = 0
    for x in data:
        if x.label == 1:
            yes += 1
        else:
            no += 1
            
    #update prediction based on yes/no counts
    leaf.prediction = 1 if yes > no else 0

    return leaf

In [16]:
#tested!
#TODO: test create_leaf_node
# test_data = []
# test_data.append(data[0])
# test_data.append(data[1])
# test_data.append(data[2])
# test_data.append(data[3])
# test_data.append(data[4])
# test_data.append(data[5])

# leeef = create_leaf_node(test_data)
# print(leeef.prediction)

Q8. Implement the `create_decision_tree` function. `max_levels` denotes the maximum height of the tree (for example if `max_levels = 1` then the decision tree will only contain the leaf node at the root. [Hint: this is where the recursion happens.]

In [17]:
def is_leaf(data):

    yes_counter = 0
    no_counter = 0
    
    #Loop through datapoints & increment yes/no counters based on label
    for x in data:
        if x.label == 1:
            yes_counter += 1
        else:
            no_counter += 1
    
    #Node is leaf if all yes's or all no's
    return True if ((yes_counter == len(data)) or (no_counter == len(data))) else False

In [18]:
def helper_build(data, max_levels, cur_levels):
    
    #base case
    if (cur_levels >= max_levels) or (is_leaf(data)):
        return create_leaf_node(data)
    else:
        best_feature, best_thresh = identify_best_split(data)
    
        #create TreeNode
        curr_node = TreeNode()
        curr_node.is_leaf = False
        curr_node.feature_idx = best_feature     
        curr_node.thresh_val = best_thresh
        
        #split data for left and right child
        left_data, right_data = split_dataset(data, best_feature, best_thresh)
    
        #set left and right child nodes by starting recursive call
        left_node = helper_build(left_data, max_levels, cur_levels + 1)
        right_node = helper_build(right_data, max_levels, cur_levels + 1)
        
        #set curr_nodes left and right node
        curr_node.left_child = left_node
        curr_node.right_child = right_node
        
        return curr_node;

In [19]:
def create_decision_tree(data, max_levels):

    cur_level = 1

    #if max levels == curlevel, root node is a leaf node
    if cur_level == max_levels:
        return create_leaf_node(data)
    
    #Get best feature & its thresh
    best_feature, best_thresh = identify_best_split(data)
    
    #Create the root node & update values
    root_node = TreeNode()
    root_node.is_leaf = False
    root_node.feature_idx = best_feature     
    root_node.thresh_val = best_thresh    
    
    #Split the data set based on thresh of chosen "best feature"
    left_data, right_data = split_dataset(data, best_feature, best_thresh)
    
    #set left and right child nodes by starting recursive call
    left_node = helper_build(left_data, max_levels, cur_level + 1)
    right_node = helper_build(right_data, max_levels, cur_level + 1)
    
    #Update root node's child values based on recursive call return value
    root_node.left_child = left_node
    root_node.right_child = right_node

    return root_node

In [20]:
# #tested!
# #TODO: test create_leaf_node
# test_data = []
# test_data.append(data[0])
# test_data.append(data[1])
# test_data.append(data[2])
# test_data.append(data[3])
# test_data.append(data[4])
# test_data.append(data[5])
# test_data.append(data[6])
# test_data.append(data[7])
# test_data.append(data[8])
# test_data.append(data[9])
# test_data.append(data[10])
# test_data.append(data[11])
# test_data.append(data[12])
# test_data.append(data[13])
# test_data.append(data[14])
# test_data.append(data[15])
# test_data.append(data[16])
# test_data.append(data[17])
# test_data.append(data[18])
# test_data.append(data[19])
# test_data.append(data[20])


# # leeef = create_leaf_node(test_data)
# root = create_decision_tree(test_data, 5)
# root.printTree()

In [22]:
# TEST CODE

## SMALL DATASET
# print('\nSMALL DATASET')
# temp_data = get_data("sample_train.txt")
# print(f'Length of dataset: {len(temp_data)}')

# print(f'Entropy of dataset: {calc_entropy(temp_data)}')

# best_gain, best_thresh = calc_best_threshold(temp_data, 3)
# print("Best thresh:", best_thresh)
# print("Best gain:", best_gain)

# feat, thresh = identify_best_split(temp_data)
# print(f"Best split: feature_index={feat}, thresh={thresh}")

# ## FULL DATASET
# print('\nFULL DATASET')
# full_data = get_data("messidor_features.txt")
# print(f'Length of dataset: {len(full_data)}')

# print(f'Entropy of dataset: {calc_entropy(full_data)}')

# best_gain, best_thresh = calc_best_threshold(full_data, 3)
# print("Best thresh:", best_thresh)
# print("Best gain:", best_gain)

# feat, thresh = identify_best_split(full_data)
# print(f"Best split: feature_index={feat}, thresh={thresh}")

# print("Create Tree:")
# temp_tree = create_decision_tree(temp_data, 10)
# temp_tree.printTree()

Q9. Given a test set, the function `calc_accuracy` returns the accuracy of the classifier. You'll use the `make_prediction` function for this.

In [23]:
def calc_accuracy(tree_root, data):
    correct = 0;
    
    #loop through all data and count number of times predicted value is correct
    for x in data:
        predicted_value = make_prediction(tree_root, x)
        actual_value = x.label
        
        if predicted_value == actual_value:
            correct +=1

    #returns correct values / total data
    return (correct / len(data))

In [24]:
# #tested :,(
# test_data = data
# print(calc_accuracy(root, test_data))
#print(calc_accuracy(root, data))

Q10. Keeping the `max_levels` parameter as 10, use 5-fold cross validation to measure the accuracy of the model. Print the accuracy of the model.

In [25]:
# edit the code here - this is just a sample to get you started
import time

d = get_data("messidor_features.txt")

acc_avg_total = 0.0

#1150/5 = 230 data per fold

start_test = 0 #inclusive range val
end_test = 230 #exclusive range val

#cross validation loop for 5-fold
for x in range (5):
    
    # the timer is just for fun! you will NOT be graded on runtime
    # start = time.time()
    
    # partition data into train_set and test_set
    train_set = d[0:start_test]
    train_set.extend(d[end_test:len(d)])
    test_set = d[start_test:end_test]

    print ('Training set size:', len(train_set))
    print ('Test set size    :', len(test_set))

    # create the decision tree
    tree = create_decision_tree(train_set, 10)

    #end = time.time()
    #print ('Time taken:', end - start)

    # calculate the accuracy of the tree
    accuracy = calc_accuracy(tree, test_set)
    print ('The accuracy on the test set is ', str(accuracy * 100.0))
    #t.printTree()
    
    #update start and end values
    start_test += 230
    end_test += 230
    
    acc_avg_total += accuracy
 
# print acc_avg_total / 5 overall accuracy   
print('Avg error over 5-fold:', (acc_avg_total / 5) * 100 )

Training set size: 920
Test set size    : 230
The accuracy on the test set is  63.04347826086957
Training set size: 920
Test set size    : 230
The accuracy on the test set is  63.04347826086957
Training set size: 920
Test set size    : 230
The accuracy on the test set is  67.82608695652173
Training set size: 920
Test set size    : 230
The accuracy on the test set is  63.47826086956522
Training set size: 920
Test set size    : 230
The accuracy on the test set is  64.34782608695652
Avg error over 5-fold: 64.34782608695652
